<a href="https://colab.research.google.com/github/Saim-Hassan786/Learn-Agentic-AI-With-OpenAI-Agents-SDK/blob/main/09-LifeCycle/LifeCycle(RunHooks_%26_AgentHooks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LifeCycle (RunHooks & AgentHooks)
**Lifecycle or hooks are callback functions that receive different lifecycle events of an Agent run.**

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GROQ_API_KEY= userdata.get('GROQ_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,set_tracing_disabled
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://groq.helicone.ai/openai/v1",
    api_key = GROQ_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

# RunHooks
**These hooks or lifecycle events are specific for the Runner Loop and they executes for every agent called in the loop.**

In [ ]:
from agents import Agent,Runner,RunHooks,RunContextWrapper,function_tool

# Initializing Runhooks from RunHooks class
class MyRunHooks(RunHooks):
  def __init__(self,agent_name):
    self.agent_name = agent_name

  async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
    print(f"Agent {self.agent_name} is Welcomed ✨")
    print(f"Agent {agent.name} started")
    print(f"Context: {context.usage}")

  async def on_agent_end(self, context: RunContextWrapper, agent: Agent,output) -> None:
    print(f"Agent {agent.name} ended")
    print(f"Context: {context.usage}")
    print(f"Output: {output}")

  async def on_handoff(self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent) -> None:
    print(f"Handoff from {from_agent.name} to {to_agent.name}")
    print(f"Context: {context.usage}")

  async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool) -> None:
    print(f"Tool {tool.name} started")
    print(f"Context: {context.usage}")

  async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool, output) -> None:
    print(f"Tool {tool.name} ended")
    print(f"Context: {context.usage}")


@function_tool
def get_weather(location:str):
  return f"The weather in {location} is sunny"

agent_spanish = Agent(
    name = "Spanish Translation Agent",
    instructions="Translate the user asked text into spanish",
    model = "llama3-70b-8192",
    handoff_description="Spanish assistant that helps in translation of asked text"
)

main_agent = Agent(
    name = "Main Agent",
    instructions="Reply to user queries **without using any tools**, unless it's a translation request.",
    model = "llama3-70b-8192",
    handoffs=[agent_spanish],
    tools=[get_weather]
)

In [ ]:
result_1 = await Runner.run(
    main_agent,
    "What is the weather in New York?",
    hooks=MyRunHooks("Saim's Agent")
)
print("======="*20)
print(f"Final Output : {result_1.final_output}")

Agent Saim's Agent is Welcomed ✨
Agent Main Agent started
Context: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
Tool get_weather started
Context: Usage(requests=1, input_tokens=1014, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=44, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1058)
Tool get_weather ended
Context: Usage(requests=1, input_tokens=1014, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=44, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1058)
Agent Main Agent ended
Context: Usage(requests=2, input_tokens=2100, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=53, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=2153)
Output: The weather in New York is sunny.
Final Output : The w

In [ ]:
result_2 = await Runner.run(
    main_agent,
    "Translate the following passage 'I am good what about you' in spanish ",
    hooks=MyRunHooks("Saim's Agent")
)
print("======="*20)
print(f"Final Output : {result_2.final_output}")

Agent Saim's Agent is Welcomed ✨
Agent Main Agent started
Context: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
Handoff from Main Agent to Spanish Translation Agent
Context: Usage(requests=1, input_tokens=1021, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=35, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1056)
Agent Saim's Agent is Welcomed ✨
Agent Spanish Translation Agent started
Context: Usage(requests=1, input_tokens=1021, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=35, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1056)
Agent Spanish Translation Agent ended
Context: Usage(requests=2, input_tokens=1132, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=95, output_tokens_details=OutputTokensDetails(reasoning

# AgentHooks
**These hooks are specific to an Agent and only executes for the specific agent when invoked.**

In [ ]:
from agents import Agent,Runner,AgentHooks,RunContextWrapper,function_tool

# Initializing Runhooks from RunHooks class
class MyAgentHooks(AgentHooks):
  def __init__(self,agent_name):
    self.agent_name = agent_name

  async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
    print(f"Agent {self.agent_name} is Welcomed ✨")
    print(f"Agent {agent.name} started")
    print(f"Context: {context.usage}")

  async def on_end(self, context: RunContextWrapper, agent: Agent,output) -> None:
    print(f"Agent {agent.name} ended")
    print(f"Context: {context.usage}")
    print(f"Output: {output}")

  async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
    print(f"Handoff from {agent.name} to {source.name}")
    print(f"Context: {context.usage}")

  async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool) -> None:
    print(f"Tool {tool.name} started")
    print(f"Context: {context.usage}")

  async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool, output) -> None:
    print(f"Tool {tool.name} ended")
    print(f"Context: {context.usage}")


@function_tool
def get_weather(location:str):
  return f"The weather in {location} is sunny"

agent_spanish = Agent(
    name = "Spanish Translation Agent",
    instructions="Translate the user asked text into spanish",
    model = "llama3-70b-8192",
    handoff_description="Spanish assistant that helps in translation of asked text",
    hooks=MyAgentHooks("Saim's Spanish Agent")
)

main_agent = Agent(
    name = "Main Agent",
    instructions="Reply to user queries **without using any tools**, unless it's a translation request.",
    model = "llama3-70b-8192",
    handoffs=[agent_spanish],
    tools=[get_weather],
    hooks=MyAgentHooks("Saim's Main Agent")
)

In [ ]:
result_with_agent_hooks = await Runner.run(
    main_agent,
    "What is the weather in New York?",
)
print("======="*20)
print(f"Final Output : {result_with_agent_hooks.final_output}")

Agent Saim's Main Agent is Welcomed ✨
Agent Main Agent started
Context: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
Tool get_weather started
Context: Usage(requests=1, input_tokens=1014, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=74, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1088)
Tool get_weather ended
Context: Usage(requests=1, input_tokens=1014, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=74, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1088)
Agent Main Agent ended
Context: Usage(requests=2, input_tokens=2100, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=83, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=2183)
Output: The weather in New York is sunny.
Final Output : 

In [ ]:
result_with_agent_hooks_2 = await Runner.run(
    main_agent,
    "Translate the following passage 'I am good what about you' in spanish ",
)
print("======="*20)
print(f"Final Output : {result_with_agent_hooks_2.final_output}")

Agent Saim's Main Agent is Welcomed ✨
Agent Main Agent started
Context: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
Handoff from Spanish Translation Agent to Main Agent
Context: Usage(requests=1, input_tokens=1021, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=43, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1064)
Agent Saim's Spanish Agent is Welcomed ✨
Agent Spanish Translation Agent started
Context: Usage(requests=1, input_tokens=1021, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=43, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=1064)
Agent Spanish Translation Agent ended
Context: Usage(requests=2, input_tokens=1134, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=103, output_tokens_details=OutputTokensDet